In [143]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import random
import re

In [144]:
class Applicant:
    def __init__(self, name, preferences_of_school, specialization):
        self.name = name
        self.specialization = specialization
        self.preferences_of_school = preferences_of_school
        self.proposal_index = 0  # Индекс в списке предпочтений, куда была сделана предложение
        self.partner = None  # Проект, с которым студент образует пару

    def make_proposal(self):
        # Метод для сделать предложение выбранным проектом из списка предпочтений
        preference_of_school = self.preferences_of_school[self.proposal_index]
        self.proposal_index += 1
        return preference_of_school

    def receive_partner(self, partner):
        # Метод для добавления проекта
        self.partner = partner

    def is_free(self):
        # Метод, возвращающий True, если студент свободен и еще не образовал пару
        return self.partner is None


class Project:
    def __init__(self, name, capacity, preferences_of_students):
        self.name = name
        self.capacity = int(capacity)  # количество мест
        self.place = None
        self.preferences_of_students = preferences_of_students  # предпочтения проекта
        self.applicants = []

    def add_applicant(self, applicant):
        # Метод для добавления студента в список принятых проектом
        self.applicants.append(applicant)

    def remove_applicant(self, applicant):
        # Метод для удаления студента из списка принятых проектом
        self.applicants.remove(applicant)

    def has_capacity(self):
        # Метод, возвращающий True, если в проекте есть свободные места
        return len(self.applicants) < self.capacity

    def prefers_applicant(self, applicant):
        # Метод, проверяющий, предпочитает ли проекта данный студент
        #return self.capacity
        #print(applicant.name, self.preferences_of_students[:self.capacity])
        return applicant.name in self.preferences_of_students[:self.capacity]

    def get_least_preferred_applicant(self):
        # Метод, возвращающий наименее предпочитаемого студента в проекте
        return self.applicants[-1]


def gale_shapley(applicants, schools, ratings):
    applicants_sort = [applicants[elem] for elem in ratings['Фамилия Имя'].tolist()]
    #return applicants
    while True:
        free_applicants = [applicant for applicant in applicants_sort if applicant.is_free()]

        if not free_applicants:
            break

        for applicant in free_applicants:
            #print(applicant.name)
            school_name = str(applicant.make_proposal())
            school = schools[school_name]

            if school.has_capacity():
                school.add_applicant(applicant)
                applicant.receive_partner(school)
                applicant.priority = applicant.preferences_of_school.index(school.name) + 1
        else:
            least_preferred_applicant = school.get_least_preferred_applicant()

            if school.prefers_applicant(applicant):
                school.remove_applicant(least_preferred_applicant)
                least_preferred_applicant.receive_partner(None)

                school.add_applicant(applicant)
                applicant.receive_partner(school)

    matches = [[applicant, applicant.partner] for applicant in applicants_sort]
    return matches

In [145]:
ratings_flutter = pd.read_excel('cur/shmr-dlja-kati.xlsx', sheet_name='Flutter').dropna()
ratings_flutter['Фамилия Имя'] = ratings_flutter['Фамилия'].str.strip() + ' ' + ratings_flutter['Имя'].str.strip()
ratings_flutter['Фамилия Имя'] = ratings_flutter['Фамилия Имя'].str.lower()
ratings_flutter

,Фамилия,Имя,Место в рейтинге,Фамилия Имя
0,Латрыгин,Арсений,1.0,латрыгин арсений
1,Смирнов,Дмитрий,2.0,смирнов дмитрий
2,Быков,Кирилл,3.0,быков кирилл
3,Султанбеков,Карим,4.0,султанбеков карим
4,Федосеев,Игорь,5.0,федосеев игорь
6,Сивцов,Павел,6.0,сивцов павел
7,Михайлов,Даниил,7.0,михайлов даниил
8,Пискунов,Александр,8.0,пискунов александр
9,Степанов,Андрей,9.0,степанов андрей
10,Дамов,Родин,10.0,дамов родин


In [146]:
ratings_android = pd.read_excel('cur/shmr-dlja-kati.xlsx', sheet_name='Android')
ratings_android['Фамилия Имя'] = ratings_android['Фамилия Имя'].str.lower()
ratings_android

,Фамилия Имя,Место в рейтинге
0,воробьев антон,1
1,задорожный глеб,2
2,беляковский михаил,3
3,языков данила,4
4,зданович иван,5
5,орел андрей,6
6,курилин антон,7
7,кудайбергенов хаби,8
8,хен данил,9
9,цыганков николай,10


In [147]:
ratings_ios = pd.read_excel('cur/shmr-dlja-kati.xlsx', sheet_name='iOS')
ratings_ios['Фамилия Имя'] = ratings_ios['Фамилия'].str.strip() + ' ' + ratings_ios['Имя'].str.strip()
ratings_ios['Фамилия Имя'] = ratings_ios['Фамилия Имя'].str.lower()
ratings_ios

,Фамилия,Имя,Место в рейтинге,Фамилия Имя
0,Ильичев,Данил,1,ильичев данил
1,Нуждин,Владимир,2,нуждин владимир
2,Ильченко,Александр,3,ильченко александр
3,Рассказов,Глеб,4,рассказов глеб
4,Федорова,София,5,федорова софия
5,Калимуллин,Тимур,6,калимуллин тимур
6,Кизельштейн,Даниил,7,кизельштейн даниил
7,Решетникова,Ангелина,8,решетникова ангелина
8,Нургалиева,Айша,9,нургалиева айша
9,Маслова,Александра,10,маслова александра


In [148]:
ratings_front = pd.read_excel('cur/Баллы ШРИ.xlsx', sheet_name='Лист1').dropna()
ratings_front['Фамилия Имя'] = ratings_front['Фамилия Имя'].str.lower().str.strip()

In [149]:
ratings_backend = pd.read_excel('cur/ШБР и рейтинг.xlsx')
ratings_backend = ratings_backend.rename({'Студент': 'Фамилия Имя'}, axis=1).sort_values(
    by='Место в рейтинге (в рамках трека)', ascending=True).dropna()
ratings_backend['Фамилия Имя'] = ratings_backend['Фамилия Имя'].str.lower()
ratings_backend['Фамилия Имя'] = ratings_backend['Фамилия Имя'].apply(
    lambda full_name: ' '.join(reversed(full_name.split(' ', 1))))

In [150]:
ratings_backend

,Фамилия Имя,Трек,Место в рейтинге (в рамках трека)
0,дробышев иван,Python,1.0
140,муратов антон,Go,1.0
49,мингарипов нияз,С++,1.0
138,хасан карим,Java,1.0
141,фадеев артём,Go,2.0
...,...,...,...
87,кортыш андрей,Java,53.0
86,цуканов михаил,Java,55.0
85,куращенко лев,Java,56.0
84,гафаров искандер,Java,59.0


In [151]:
ratings_python = ratings_backend[ratings_backend['Трек'] == 'Python']
ratings_c = ratings_backend[ratings_backend['Трек'] == 'С++']
ratings_java = ratings_backend[ratings_backend['Трек'] == 'Java']
ratings_go = ratings_backend[ratings_backend['Трек'] == 'Go']
ratings_java

,Фамилия Имя,Трек,Место в рейтинге (в рамках трека)
138,хасан карим,Java,1.0
137,абрамов александр,Java,2.0
136,горюнов александр,Java,3.0
135,суетов денис,Java,4.0
134,горелов максим,Java,5.0
133,загретдинов ильдар,Java,6.0
132,реськов кирилл,Java,7.0
131,безруков павел,Java,8.0
130,садовский максим,Java,9.0
129,абрашков вадим,Java,10.0


In [152]:
df = pd.read_excel('cur/Студенты по локациям 2023 (4).xlsx', sheet_name='Тайминг 24-25 июля')
df = df.drop(index=0)
df['Хаб'].replace('Москва + возможность сформировать часть команды в КЗ', 'Москва', inplace=True)
df['Хаб'] = df['Хаб'].apply(lambda x: x.strip())
selected_columns = ['Бэкенд Python', 'Бэкенд Go', 'Бэкенд C++', 'Бэкенд Java', 'Фронт', 'Мобилка iOS',
                    'Мобилка Android', 'Мобилка Flutter']
df[selected_columns] = df[selected_columns].apply(pd.to_numeric, errors='coerce')
df[selected_columns] = df[selected_columns].fillna(df[selected_columns])
selected_columns_2 = ['Тестировщик', 'Дизайнер']
#df[selected_columns_2] = df[selected_columns_2].applymap(lambda x: 1.0 if x.lower() == 'нужен' else np.nan)
df['Дизайнер'] = df['Дизайнер'].apply(lambda x: 2 if str(x).lower() == 'нужен' else np.nan)
df.at[9, 'Дизайнер'] = 3
df['Тестировщик'] = df['Тестировщик'].apply(lambda x: 1 if str(x).lower() == 'нужен' else np.nan)

In [153]:
df['Тестировщик'].sum()

34.0

In [154]:
projects_python = {}
projects_go = {}
projects_c = {}
projects_java = {}

projects_front = {}

projects_ios = {}
projects_android = {}
projects_flutter = {}

projects_qa = {}
projects_design = {}

for index, row in df.iterrows():
    place = row['Хаб']

    if pd.notna(row['Бэкенд Python']):
        name = row['Название проекта'].strip()
        name += ' # Python'
        project = Project(name=name, capacity=row['Бэкенд Python'],
                          preferences_of_students=ratings_python['Фамилия Имя'].tolist())
        project.place = place
        projects_python[name] = project

    if pd.notna(row['Бэкенд Go']):
        name = row['Название проекта'].strip()
        name += ' # Go'
        project = Project(name=name, capacity=row['Бэкенд Go'],
                          preferences_of_students=ratings_go['Фамилия Имя'].tolist())
        project.place = place
        projects_go[name] = project

    if pd.notna(row['Бэкенд C++']):
        name = row['Название проекта'].strip()
        name += ' # C++'
        project = Project(name=name, capacity=row['Бэкенд C++'],
                          preferences_of_students=ratings_c['Фамилия Имя'].tolist())
        project.place = place
        projects_c[name] = project

    if pd.notna(row['Бэкенд Java']):
        name = row['Название проекта'].strip()
        name += ' # Java'
        project = Project(name=name, capacity=row['Бэкенд Java'],
                          preferences_of_students=ratings_java['Фамилия Имя'].tolist())
        project.place = place
        projects_java[name] = project

    if pd.notna(row['Фронт']):
        name = row['Название проекта'].strip()
        name += ' # Фронт'
        project = Project(name=name, capacity=row['Фронт'],
                          preferences_of_students=ratings_front['Фамилия Имя'].tolist())
        project.place = place
        projects_front[name] = project

    if pd.notna(row['Мобилка iOS']):
        name = row['Название проекта'].strip()
        name += ' # iOS'
        project = Project(name=name, capacity=row['Мобилка iOS'],
                          preferences_of_students=ratings_ios['Фамилия Имя'].tolist())
        project.place = place
        projects_ios[name] = project

    if pd.notna(row['Мобилка Android']):
        name = row['Название проекта'].strip()
        name += ' # Android'
        project = Project(name=name, capacity=row['Мобилка Android'],
                          preferences_of_students=ratings_android['Фамилия Имя'].tolist())
        project.place = place
        projects_android[name] = project

    if pd.notna(row['Мобилка Flutter']):
        name = row['Название проекта'].strip()
        name += ' # Flutter'
        project = Project(name=name, capacity=row['Мобилка Flutter'],
                          preferences_of_students=ratings_flutter['Фамилия Имя'].tolist())
        project.place = place
        projects_flutter[name] = project

    if pd.notna(row['Тестировщик']):
        name = row['Название проекта'].strip()
        name += ' # Тестировщик'
        project = Project(name=name, capacity=row['Тестировщик'], preferences_of_students=[])
        project.place = place
        projects_qa[name] = project

    if pd.notna(row['Дизайнер']):
        name = row['Название проекта'].strip()
        name += ' # Дизайнер'
        project = Project(name=name, capacity=row['Дизайнер'], preferences_of_students=[])
        project.place = place
        projects_design[name] = project

In [155]:
len(projects_qa)

34

In [156]:
#assert sum(value.capacity for value in projects_flutter.values()) == len(ratings_flutter)
assert sum(value.capacity for value in projects_ios.values()) == len(ratings_ios)
assert sum(value.capacity for value in projects_android.values()) == len(ratings_android)

#assert sum(value.capacity for value in projects_front.values()) == len(ratings_front)

assert sum(value.capacity for value in projects_python.values()) == len(ratings_python)
#assert sum(value.capacity for value in projects_c.values()) == len(ratings_c)
assert sum(value.capacity for value in projects_java.values()) == len(ratings_java)
assert sum(value.capacity for value in projects_go.values()) == len(ratings_go)

In [157]:
#priority = pd.read_excel('cur/Распределение_проектов_по_разработчикам.xlsx', sheet_name='Приоритеты студентов', header=5)
priority = pd.read_excel('cur/Распределение проектов по разработчикам.xlsx', sheet_name='Приоритеты студентов',
                         header=5)

F:\python3.10\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [158]:
priority

,С кем ты хочешь быть на одном проекте (не забудьте указать все одинаковые приоритеты),Студент,Школа,Направление,Приоритет 1,Приоритет 2,Приоритет 3,Приоритет 4,Приоритет 5,Приоритет 6,...,Приоритет 43,Приоритет 44,Приоритет 45,Приоритет 46,Приоритет 47,Приоритет 48,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,"Матвиенко Назар, Мезенин Олег",Ковалюк Алексей,ШБР,Python,13. Я команда,15. Интерфейс для администрирования каналов ко...,14. Ачивница 2.0,44. Автоматизация проверки работ на направлени...,38. Сетевая реферальная программа,24. Командировки в Go для бизнеса. ЛК Сотрудника,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
1,"Дергунов Игорь, Гоним Манар, Царапкин Дмитрий...",Суббота Анастасия,ШБР,Python,19. Эзотерика в Алисе,14. Ачивница 2.0,18. Умная система ценообразования заданий для ...,37. Софт для эффективного управления автомобилями,16. Расширение географии активных исполнителей...,15. Интерфейс для администрирования каналов ко...,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
2,NaN,Глазачева Анна,ШБР,Python,19. Эзотерика в Алисе,14. Ачивница 2.0,48. Плафторма Яндекс Практикума,44. Автоматизация проверки работ на направлени...,38. Сетевая реферальная программа,13. Я команда,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
3,NaN,Лапа Анна,ШБР,Python,44. Автоматизация проверки работ на направлени...,13. Я команда,48. Плафторма Яндекс Практикума,14. Ачивница 2.0,43. Геймификация выдачи офферов пользователям,38. Сетевая реферальная программа,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
4,NaN,Антонов Артём,ШБР,Python,14. Ачивница 2.0,43. Геймификация выдачи офферов пользователям,19. Эзотерика в Алисе,18. Умная система ценообразования заданий для ...,13. Я команда,15. Интерфейс для администрирования каналов ко...,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,NaN,Пашин Максим,ШБР,Go,Биддер для Маркета,Тариф Magic / поручения курьеру в международно...,Елочка 2.0,MatchMaster: инновационный матчинг абитуриенто...,Создать конструктор креативов для партнеров,Алиса: навыки для офисов,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,NaN,Перелыгин Иван,ШБР,Go,34. Заправки 3.0 (1 - геймификация),35. Заправки 3.0 (2 - онбординг),27. Алиса: навыки для офисов,57. Биддер для Маркета,58. Елочка 2.0,56. Создать конструктор креативов для партнеров,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
applicants_python = {}
applicants_go = {}
applicants_c = {}
applicants_java = {}

applicants_front = {}

applicants_ios = {}
applicants_android = {}
applicants_flutter = {}

applicants_qa = {}
applicants_design = {}

random.seed(0)
for index, row in priority.iterrows():
    name = row['Студент']
    if not pd.notna(name):
        continue
    name = re.sub(r"[^а-яёЁА-Яa-zA-ZӨөмірзақ\s]", "", name).strip()
    if isinstance(name, str):
        name = name.lower()
        name = name.replace(' ', ' ')
    specialization = row['Направление']
    priority_values = []
    for i in range(1, 48):
        column_name = f"Приоритет {i}"
        # Извлекаем значения из колонки
        value = row[column_name]
        # Добавляем непустые и не "-" значения в список priority_values
        if pd.notna(value) and value != "-":
            value = value[value.index(". ") + 2::] if '. ' in value else value
        result = f'{value.strip()} # {specialization}' if pd.notna(value) and value != "-" else None
        priority_values.append(result if result not in priority_values else None)

    if specialization == 'Python':
        remaining_values = list(set(projects_python) - set(priority_values))
        random.shuffle(remaining_values)
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        print(name, priority_values)
        #print(len(priority_values))
        #print(priority_values, projects_python.keys())
        #print(len(priority_values))
        assert len(priority_values) == len(projects_python), f'{len(priority_values)}, {len(projects_python)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_python[name] = applicant
    elif specialization == 'C++':
        remaining_values = list(set(projects_c) - set(priority_values))
        random.shuffle(remaining_values)
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        assert len(priority_values) == len(projects_c), f'{len(priority_values)}, {len(projects_c)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_c[name] = applicant
    if specialization == 'Java':
        print(name, priority_values)
        remaining_values = list(set(projects_java) - set(priority_values))
        random.shuffle(remaining_values)
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        assert len(priority_values) == len(projects_java), f'{len(priority_values)}, {len(projects_java)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_java[name] = applicant
    elif specialization == 'Go':
        remaining_values = list(set(projects_go) - set(priority_values))
        #print(remaining_values)
        random.shuffle(remaining_values)
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        #print(len(priority_values), len(projects_go))
        print(name, priority_values)
        assert len(priority_values) == len(projects_go), f'{len(priority_values)}, {len(projects_go)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_go[name] = applicant

    if specialization == 'iOS':
        remaining_values = list(set(projects_ios) - set(priority_values))
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        assert len(priority_values) == len(projects_ios), f'{len(priority_values)}, {len(projects_ios)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_ios[name] = applicant
    elif specialization == 'Flutter':
        remaining_values = list(set(projects_flutter) - set(priority_values))
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        assert len(priority_values) == len(projects_flutter), f'{len(priority_values)}, {len(projects_flutter)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_flutter[name] = applicant
    if specialization == 'Android':
        remaining_values = list(set(projects_android) - set(priority_values))
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        assert len(priority_values) == len(projects_android), f'{len(priority_values)}, {len(projects_android)}'

        applicant = Applicant(name, priority_values, specialization)
        applicants_android[name] = applicant
    elif specialization == 'Фронт':
        print(name, value)
        remaining_values = list(set(projects_front) - set(priority_values))
        print(priority_values)
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        assert len(priority_values) == len(projects_front)

        applicant = Applicant(name, priority_values, specialization)
        applicants_front[name] = applicant

    # elif specialization == 'Тестировщик':
    #     remaining_values = list(set(projects_qa) - set(priority_values))
    #     priority_values += remaining_values
    #     priority_values = list(filter(lambda x: x is not None, priority_values))
    #     assert len(priority_values) == len(projects_qa)
    #
    #     applicant = Applicant(name, priority_values, specialization)
    #     applicants_qa[name] = applicant
    # elif specialization == 'Дизайнер':
    #     remaining_values = list(set(projects_design) - set(priority_values))
    #     priority_values += remaining_values
    #     priority_values = list(filter(lambda x: x is not None, priority_values))
    #     assert len(priority_values) == len(projects_design)
    #
    #     applicant = Applicant(name, priority_values, specialization)
    #     applicants_design[name] = applicant
    #
    #
    # priority_values += remaining_values
    # priority_values = list(filter(lambda x: x is not None, priority_values))

ковалюк алексей ['Я команда # Python', 'Интерфейс для администрирования каналов коммуникаций - "Анонсилка 2.0" # Python', 'Ачивница 2.0 # Python', 'Автоматизация проверки работ на направлении дизайна # Python', 'Сетевая реферальная программа # Python', 'Командировки в Go для бизнеса. ЛК Сотрудника # Python', 'Плафторма Яндекс Практикума # Python', 'Командировки в Go для бизнеса. Адаптация для Энтерпрайз-клиентов # Python', 'Расширение географии активных исполнителей заданий, работающих удаленно # Python', 'Геймификация выдачи офферов пользователям # Python', 'Эзотерика в Алисе # Python', 'Комьюнити для пешеходов # Python', 'Софт для эффективного управления автомобилями # Python', 'Warehouse constructor # Python', 'Умная система ценообразования заданий для географически распределенных исполнителей # Python', 'Онлайн-тренировки для команд по спортивному программированию # Python']
суббота анастасия ['Эзотерика в Алисе # Python', 'Ачивница 2.0 # Python', 'Умная система ценообразования зад

In [160]:
applicants_ios['рассказов глеб'] = Applicant('рассказов глеб',
                                             ['Мобильный кабинет разработчика на платформе Яндекс.Игр (1) # iOS',
                                              'Мобильный кабинет разработчика на платформе Яндекс.Игр (2) # iOS',
                                              'Мультимодальность в Yandex Go # iOS',
                                              'Продукт для бизнеса внутри Яндекс Go # iOS',
                                              'Вырастить кол-во доставок на следующий день в Яндекс Go # iOS',
                                              'Букинг Такси (B2B, B2C) # iOS',
                                              'Тариф Magic / поручения курьеру в международной Доставке # iOS',
                                              'Менеджер артиста: Продвижение # iOS',
                                              'Генеративная модель в подборе тура # iOS',
                                              'Умная камера для незрячих пользователей # iOS',
                                              'Summary путешествия # iOS', 'Музыка для кино # iOS',
                                              'Менеджер артиста: Создание музыки # iOS',
                                              'Менеджер артиста: MusicID # iOS'], specialization='iOS')

In [161]:
priority_test = pd.read_excel('cur/Распределение проектов Практикум.xlsx', sheet_name='Приоритеты студентов',
                         header=5)
priority_test

F:\python3.10\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Студент,Школа,Направление,Приоритет 1,Приоритет 2,Приоритет 3,Приоритет 4,Приоритет 5,Приоритет 6,Приоритет 7,...,Приоритет 39,Приоритет 40,Приоритет 41,Приоритет 42,Приоритет 43,Приоритет 44,Приоритет 45,Приоритет 46,Приоритет 47,Приоритет 48
0,Валерия Безродняя,Практикум,Тестировщик,49. Бренд-платформа для обучения по геймдеву о...,14. Ачивница 2.0,32. Елочка 2.0,33. Яндекс Афиша,40. Потеряшкин,39. Биддер для Маркета,12. Геймификация погоды,...,-,-,-,-,-,-,-,-,-,-
1,Алина Кударенко,Практикум,Тестировщик,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-,-,-,-,-,-,-,-,-,-
2,Яна Кочеткова,Практикум,Тестировщик,40. Потеряшкин,33. Яндекс Афиша,56. Мобильный кабинет разработчика на платформ...,57. Мобильный кабинет разработчика на платформ...,16. Расширение географии активных исполнителей...,32. Елочка 2.0,27. Ad/Ed Tech,...,-,-,-,-,-,-,-,-,-,-
3,Константин Шарпань,Практикум,Тестировщик,33. Яндекс Афиша,13. Я команда,36. Warehouse constructor,14. Ачивница 2.0,47. Продукт для бизнеса внутри Яндекс Go,31. Какой ты партнёр в Я.Еде,40. Потеряшкин,...,-,-,-,-,-,-,-,-,-,-
4,Илона Кочарян,Практикум,Тестировщик,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Гринченко Мария Вячеславовна,Практикум,Дизайнер,48. Плафторма Яндекс Практикума,44. Автоматизация проверки работ на направлени...,14. Ачивница 2.0,43. Геймификация выдачи офферов пользователям,15. Интерфейс для администрирования каналов ко...,32. Елочка 2.0,49. Бренд-платформа для обучения по геймдеву о...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,Юлия Шавидзе,Практикум,Дизайнер,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,Никифорова Изабелла Александровна,Практикум,Дизайнер,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Анастасия Муктасимова,Практикум,Дизайнер,56. Мобильный кабинет разработчика на платформ...,44. Автоматизация проверки работ на направлени...,15. Интерфейс для администрирования каналов ко...,14. Ачивница 2.0,48. Плафторма Яндекс Практикума,40. Потеряшкин,49. Бренд-платформа для обучения по геймдеву о...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
for index, row in priority_test.iterrows():
    name = row['Студент']
    if not pd.notna(name):
        continue
    #name = re.sub(r"[^а-яёЁА-Яa-zA-ZӨөмірзақ\s]", "", name).strip()
    if isinstance(name, str):
        name = name.lower()
        name = name.replace(' ', ' ')
    specialization = row['Направление']
    priority_values = []
    for i in range(1, 48):
        column_name = f"Приоритет {i}"
        # Извлекаем значения из колонки
        value = row[column_name]
        # Добавляем непустые и не "-" значения в список priority_values
        if pd.notna(value) and value != "-":
            value = value[value.index(". ") + 2::] if '. ' in value else value
        result = f'{value.strip()} # {specialization}' if pd.notna(value) and value != "-" else None
        priority_values.append(result if result not in priority_values else None)

    if specialization == 'Тестировщик':
        remaining_values = list(set(projects_qa) - set(priority_values))
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        #assert len(priority_values) == len(projects_qa)

        applicant = Applicant(name, priority_values, specialization)
        applicants_qa[name] = applicant
    elif specialization == 'Дизайнер':
        remaining_values = list(set(projects_design) - set(priority_values))
        priority_values += remaining_values
        priority_values = list(filter(lambda x: x is not None, priority_values))
        #assert len(priority_values) == len(projects_design)

        applicant = Applicant(name, priority_values, specialization)
        applicants_design[name] = applicant

In [163]:
len(applicants_qa)

34

In [164]:
ratings_test = pd.read_excel('cur/Список участников в Летних школах.xlsx', sheet_name='Лист2').sort_values(by='Возраст', ascending=False)
ratings_qa = ratings_test.rename({'ФИ': 'Фамилия Имя'}, axis=1)
ratings_qa['Фамилия Имя'] = ratings_qa['Фамилия Имя'].str.lower()
ratings_qa['Фамилия Имя']

9          наталья буданова
13           анна прохорова
17         наталья миронова
12           василий бычков
25           галина хотяева
7        евгения глазовская
15       вероника гончарова
28            юлия янтурина
21          сергей самойлов
24           евгений рузлёв
18        владислав сазонов
30           данил бардюков
6          екатерина полова
4             илона кочарян
3        константин шарпань
29         александр жвакин
26             артем иванов
14            павел коровин
20         алексей жеребцов
2             яна кочеткова
27            нагиев ровшан
22        надежда коршикова
16            павел коровин
5         владимир капралов
1           алина кударенко
0         валерия безродняя
19            анна нарыжная
11             илья морозов
10         валентина расули
8     валерия спитбатталова
23      ангелина григорьева
31           татьяна ларина
32        никита самойленко
33       алина красноперова
Name: Фамилия Имя, dtype: object

In [165]:
ratings_design = pd.read_excel('cur/рейтинг дизайнеры.xlsx').rename({'Имя': 'Фамилия Имя'}, axis=1)
ratings_design['Фамилия Имя'] = ratings_design['Фамилия Имя'].str.lower()
ratings_design

,Фамилия Имя,Рейтинг
0,тимофей декало,1
1,дарья игоревна шведова,2
2,полина селезнева,3
3,казакова марина александровна,4
4,исаева елена борисовна,5
...,...,...
91,жукова кристина сергеевна,92
92,антонов данил павлович,93
93,глушков никита олегович,94
94,юлия шавидзе,95


In [166]:
# matches_design = gale_shapley(applicants_design, projects_design, ratings_design)
# #matches_design
# ans_design = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_design]).rename(
#     {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
# ans_design.to_excel('cur/Практикум.xlsx', index=False, sheet_name='Дизайнеры')

In [167]:
matches_qa = gale_shapley(applicants_qa, projects_qa, ratings_qa)
#matches_qa
with pd.ExcelWriter('cur/Практикум.xlsx', engine='openpyxl', mode='a') as writer:
    ans_qa = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_qa]).rename({0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
    # Добавляем DataFrame в новый лист
    ans_qa.to_excel(writer, sheet_name='QA', index=False)

In [168]:
matches_python = gale_shapley(applicants_python, projects_python, ratings_python)
#matches_python
ans_python = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_python]).rename(
    {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
ans_python.to_excel('cur/ШБР.xlsx', index=False, sheet_name='Python')

In [169]:
matches_c = gale_shapley(applicants_c, projects_c, ratings_c)
#matches_c
with pd.ExcelWriter('cur/ШБР.xlsx', engine='openpyxl', mode='a') as writer:
    ans_c = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_c]).rename({0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
    # Добавляем DataFrame в новый лист
    ans_c.to_excel(writer, sheet_name='C++', index=False)

In [170]:
matches_java = gale_shapley(applicants_java, projects_java, ratings_java)
#matches_java
with pd.ExcelWriter('cur/ШБР.xlsx', engine='openpyxl', mode='a') as writer:
    ans_java = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_java]).rename(
        {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
    # Добавляем DataFrame в новый лист
    ans_java.to_excel(writer, sheet_name='Java', index=False)

In [171]:
matches_go = gale_shapley(applicants_go, projects_go, ratings_go)
#matches_go
with pd.ExcelWriter('cur/ШБР.xlsx', engine='openpyxl', mode='a') as writer:
    ans_go = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_go]).rename(
        {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
    # Добавляем DataFrame в новый лист
    ans_go.to_excel(writer, sheet_name='Go', index=False)

In [172]:
matches_android = gale_shapley(applicants_android, projects_android, ratings_android)
#matches_android
ans_android = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_android]).rename(
    {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
ans_android.to_excel('cur/ШМР.xlsx', index=False, sheet_name='Android')

In [173]:
matches_ios = gale_shapley(applicants_ios, projects_ios, ratings_ios)
#matches_ios
with pd.ExcelWriter('cur/ШМР.xlsx', engine='openpyxl', mode='a') as writer:
    ans_ios = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_ios]).rename(
        {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
    # Добавляем DataFrame в новый лист
    ans_ios.to_excel(writer, sheet_name='iOS', index=False)

In [174]:
matches_flutter = gale_shapley(applicants_flutter, projects_flutter, ratings_flutter)
#matches_flutter
with pd.ExcelWriter('cur/ШМР.xlsx', engine='openpyxl', mode='a') as writer:
    ans_flutter = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_flutter]).rename(
        {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
    # Добавляем DataFrame в новый лист
    ans_flutter.to_excel(writer, sheet_name='Flutter', index=False)

In [175]:
matches_front = gale_shapley(applicants_front, projects_front, ratings_front)
#matches_front
ans_front = pd.DataFrame([(elem[0].name, elem[1].name) for elem in matches_front]).rename(
    {0: 'Фамилия Имя', 1: 'Номер полученного проекта'}, axis=1)
ans_front.to_excel('cur/Фронт.xlsx', index=False, sheet_name='Фронт')

In [176]:
def metrics(matches):
    ind_of_recieved_projects = [matches[i][0].preferences_of_school.index(matches[i][1].name) + 1 for i in
                                range(len(matches))]
    return f'среднее место проекта у студентов: {np.mean(ind_of_recieved_projects):.3f}'


def plot(matches, specialization):
    ind_of_recieved_projects = [matches[i][0].preferences_of_school.index(matches[i][1].name) + 1 for i in
                                range(len(matches))]

    x = range(1, len(ind_of_recieved_projects) + 1)
    y = ind_of_recieved_projects

    plt.scatter(x, y)
    plt.xlabel('Номер по баллам')
    #plt.ylim(0, 34)
    plt.ylabel('Место проекта у студента')
    plt.title('График ' + specialization)
    plt.show()

    return ind_of_recieved_projects

In [177]:
plot(matches_design, 'Дизайнеры')
print(metrics(matches_design))

NameError: name 'matches_design' is not defined

In [ ]:
plot(matches_python, 'Python')
print(metrics(matches_python))
plot(matches_c, 'C++')
print(metrics(matches_c))
plot(matches_java, 'Java')
print(metrics(matches_java))
plot(matches_go, 'Go')
print(metrics(matches_go))

In [ ]:
plot(matches_android, 'Android')
print(metrics(matches_android))
plot(matches_ios, 'iOS')
print(metrics(matches_ios))
plot(matches_flutter, 'Flutter')
print(metrics(matches_flutter))

In [ ]:
plot(matches_front, 'Фронт')
print(metrics(matches_front))